In [ ]:
%load_ext autoreload
%autoreload 2
#%pylab inline

In [ ]:
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'

In [ ]:
import tensorflow as tf
import numpy as np
import itertools
import keras
import gym

from tqdm import tqdm
from wrappers.envs import AtariStackFrames
from utils.ae import AEConvKeras

In [ ]:
model = keras.models.load_model('models/model_20170209-152142/model_epoch49.h5')

In [ ]:
env = AtariStackFrames(gym.make('MsPacman-v0'), rgb=True)

In [ ]:
def collect_data(env=None, n_episodes=10, max_steps=5000):
    states = []
    for i in tqdm(range(n_episodes)):
        state = env.reset()
        states.append(state)
        terminal = False
        step = 0
        while not terminal and step < max_steps:
            action = env.action_space.sample()
            state, reward, terminal, _ = env.step(action)
            states.append(state)
            step += 1
    return np.array(states)

def run_episode(env=None, agent=None, max_steps=5000):
    state = env.reset()
    total_reward = 0.
    terminal = False
    step = 0
    while not terminal and step < max_steps:
        action = agent.predict(np.expand_dims(state, 0)).argmax(axis=1)
        #action = env.action_space.sample()
        state, reward, terminal, _ = env.step(action)
        total_reward += reward
        step += 1
    return total_reward

In [ ]:
res = []
for i in xrange(20):
    r = run_episode(env, model)
    print r
    res.append(r)
print 'mean: {:.3f}'.format(np.array(res).mean())

In [ ]:
[x.shape for x in model.layers[-1].get_weights()]

In [ ]:
model.layers[-1].trainable

In [ ]:
tf.Variable(model.trainable_weights[0], trainable=False)